In [ ]:
!pip install autokeras

In [ ]:
#all the imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import tensorflow as tf
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import string
import re
print(tf.__version__)

In [ ]:
#Load the data
data= pd.read_json("../input/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json", lines = True)

# **preprocessing**

In [ ]:
data.head()

In [ ]:
data.drop(columns="article_link",axis='columns', inplace=True)
#useless column for our case 

In [ ]:
print("Null Values\n",data.isnull().sum())
print("\nData Shape: " , data.shape)

# **Preprocessing**

In [ ]:
data["headline"]=data["headline"].str.lower()
data.head()

In [ ]:
def remove_stopwords(txt):
    STOPWORDS = set(stopwords.words('english'))
    return " ".join([word for word in str(txt).split() if word not in STOPWORDS])
data["headline"] = data["headline"].apply(lambda txt: remove_stopwords(txt))
data.head()

In [ ]:
def remove_punctuation(txt):
    PUNCT_TO_REMOVE = string.punctuation
    return txt.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

data["headline"] = data["headline"].apply(lambda txt: remove_punctuation(txt))
data.head()

In [ ]:
def remove_brackets_text(txt):
    return  re.sub('\[.*?\]', '', txt)
data["headline"] = data["headline"].apply(lambda txt: remove_brackets_text(txt))
data.head()

In [ ]:
def remove_words_with_numbers(txt):
    return re.sub('\w*\d\w*', '', txt)
data["headline"] = data["headline"].apply(lambda txt: remove_words_with_numbers(txt))
data.head()

In [ ]:
# def lemmatize_words(txt):
#     lemmatizer = WordNetLemmatizer()
#     wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
#     pos_tagged_text = pos_tag(txt.split())
#     return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

# data["headline"] = data["headline"].apply(lambda txt: lemmatize_words(txt))
# data.head()

In [ ]:
X=data["headline"].tolist()
y=data["is_sarcastic"].tolist()

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,shuffle=True)

In [ ]:
x_train=np.array(X_train)
x_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

# **Making the model**

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextBlock(block_type="ngram")(input_node)
output_node = ak.ClassificationHead()(output_node)

In [ ]:
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node
)

# **Training the model and evaluation**

In [ ]:
clf.fit(x_train,y_train,epochs=10,validation_split=0.2)

In [ ]:
clf.evaluate(x_test, y_test)

# **Testing the model**